In [23]:
import pandas as pd
import numpy as np
import time

from google import genai
from google.genai import types, errors
from pydantic import BaseModel, Field
from typing import overload, Union

not_in_new_df = pd.read_excel("output/papers_not_in_new.xlsx")

two_banks = ("ING", "HSBC")

two_banks_df = not_in_new_df[not_in_new_df["company_name"].isin(two_banks)][["company_name", "title", "description", "paper_url"]]

In [24]:
class ResearchPapersRelevanceOutput(BaseModel):
    relevance: str = Field(
        ...,
        description="""
         A value, yes, no or unsure about how relevant a paper is to ai
       """
    )  

relevance_papers_prompt = """
You are an expert in AI and you will be given a list of research papers. Your task is to determine if the paper is relevant to AI or not. Output a one word answer regrading its relevance: Yes, No or Unsure.
"""

In [25]:
user_prompt = """
        Here is your input:
        {text}
        """
# name of the gemini model we are using
model_name = "google/gemini-2.5-flash"

# gemini client and configuration
google_genai_client = genai.Client(
    vertexai=True,
    project="evident-data-dev",
    location="europe-west1",
)

# setup the model
model_configuration = types.GenerateContentConfig(
    system_instruction=relevance_papers_prompt,
    response_mime_type="application/json",
    response_schema=ResearchPapersRelevanceOutput,
)

def classify_dataframe(text_df: pd.DataFrame, text_column: str, identifiable_column: str):
        """
        Takes a pandas DataFrame of potential use cases and
        dds the required metadata so it can be added to the use case tracker
        Args:
            text_df (pd.DataFrame): press releases
            text_column str: name of the column containing the body of text to use
        Returns:
            A pandas DataFrame containing the LLM JSON output as columns
        """
        classification_list = []
        for i in range(len(text_df)):
            if (i+1) % 100 == 0:
                print("Waiting 1min 30s")
                time.sleep(90)
            try:
                response = google_genai_client.models.generate_content(
                    model=model_name,
                    config=model_configuration,
                    contents=user_prompt.format(text=text_df[text_column].iloc[i]),
                )
            except errors.APIError as e:
                raise ValueError(f"Code:{e.code}", "\n", f"Message: {e.message}")
                
            classification_list.append([i,
                                        text_df[identifiable_column].iloc[i],
                                        text_df[text_column].iloc[i],
                                        response.parsed.relevance
                                        ])
            
            if (i + 1) % 50 == 0 or (i + 1) == len(text_df):
                print(f"Progress: {(i + 1) / len(text_df):.2%}")
        
        return pd.DataFrame(classification_list, columns=[
            "index",
            identifiable_column,
            "body",
            "relevance"
                ])

In [26]:
relevance_df = classify_dataframe(two_banks_df, "description", "paper_url")

relevance_df

c:\Users\lucas\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


ValueError: ('Code:403', '\n', "Message: Permission 'aiplatform.endpoints.predict' denied on resource '//aiplatform.googleapis.com/projects/evident-data-dev/locations/europe-west1/publishers/google/models/gemini-2.5-flash' (or it may not exist).")